In [1]:
import os 
os.getcwd()

'C:\\Users\\DJ COMPUTERS'

In [3]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_excel(r"C:\Users\DJ COMPUTERS\OneDrive\Desktop\SplitiFy\budgetwise_finance_dataset.xlsx")
data.head()

,date,category,amount
0,2021-01-01 00:00:00,Educaton,3888
1,2021-01-01 00:00:00,rent,649
2,2021-01-01 00:00:00,Freelance,13239
3,2021-01-01 00:00:00,Fod,6299
4,2021-01-01 00:00:00,entertainment,2287


In [7]:
data.shape

(15900, 3)

In [9]:
data.isna().sum()

date        486
category    285
amount      291
dtype: int64

# Cleaning & Modelling th 'category' column

In [12]:
data['category'].value_counts()

category
FOOD             583
Food             579
RENT             574
Rent             571
Foods            567
food             567
Others           554
Fod              535
rent             534
Foodd            525
Rentt            517
Rnt              516
Freelance        490
Investment       482
Bonus            442
Salary           430
Travel           422
Traval           411
TRAVEL           403
travel           381
Travl            368
Entertain        349
Entrtnmnt        340
Utility          330
Utilties         329
entertainment    327
Entertainment    319
Utilities        317
utilities        313
Utlities         300
education        284
Education        259
Educaton         258
EDU              235
health           159
Helth            157
Health           156
HEALTH           151
Savings          110
SAVINGS          101
savings           99
Saving            85
Other             50
others            48
Misc              45
OTHERS            43
Name: count, dtype: int64

In [14]:
from difflib import get_close_matches

def clean_category_column(df, column_name="Category", similarity_threshold=0.8):
   
    df[column_name] = df[column_name].astype(str).str.strip().str.lower()
    
    base_categories = [
        "food", "rent", "travel", "entertainment", "utility",
        "education", "health", "savings", "others", "bonus",
        "investment", "salary", "freelance", "misc"
    ]
    
    mapping = {}
    for cat in df[column_name].unique():
        match = get_close_matches(cat, base_categories, n=1, cutoff=similarity_threshold)
        mapping[cat] = match[0] if match else cat  # keep original if no close match
    
    df[column_name] = df[column_name].map(mapping)
    
    return df


In [16]:
# Clean category column
data = clean_category_column(data, column_name="category")

# Check results
print(data["category"].value_counts())

category
food             3356
rent             2712
travel           1985
entertainment    1335
education         801
others            695
utilities         630
health            623
freelance         490
investment        482
bonus             442
salary            430
savings           395
utility           330
utilties          329
utlities          300
nan               285
edu               235
misc               45
Name: count, dtype: int64


In [18]:
def finalize_category_cleanup(df, column_name="Category"):
    """
    Cleans and groups category names into standardized labels.
    """
    df[column_name] = df[column_name].astype(str).str.strip().str.lower()
    
    df[column_name] = df[column_name].replace("nan", np.nan)
    
    # Define manual category mapping
    category_mapping = {
        # Education group
        "edu": "education",
        "educaton": "education",
        
        # Utilities group
        "utility": "utilities",
        "utilties": "utilities",
        "utlities": "utilities",
        
        # Others/Misc group
        "misc": "others",
        "other": "others",
        "others": "others"
    }
    
    # Apply mapping
    df[column_name] = df[column_name].replace(category_mapping)
    
    # Replace NaN with 'others'
    df[column_name] = df[column_name].fillna("others")
    
    # Capitalize the final category names
    df[column_name] = df[column_name].str.capitalize()
    
    return df


In [20]:
# Apply function
data = finalize_category_cleanup(data, "category")

print(data)
print("\nCategory counts:")
print(data["category"].value_counts())

                      date       category    amount
0      2021-01-01 00:00:00      Education      3888
1      2021-01-01 00:00:00           Rent       649
2      2021-01-01 00:00:00      Freelance     13239
3      2021-01-01 00:00:00           Food      6299
4      2021-01-01 00:00:00  Entertainment      2287
...                    ...            ...       ...
15895                  NaN           Rent      7335
15896                  NaN      Education      4353
15897                  NaN           Food      1048
15898                  NaN         Others    621176
15899                  NaN      Utilities  5459 INR

[15900 rows x 3 columns]

Category counts:
category
Food             3356
Rent             2712
Travel           1985
Utilities        1589
Entertainment    1335
Education        1036
Others           1025
Health            623
Freelance         490
Investment        482
Bonus             442
Salary            430
Savings           395
Name: count, dtype: int64


In [22]:
data.head()

,date,category,amount
0,2021-01-01 00:00:00,Education,3888
1,2021-01-01 00:00:00,Rent,649
2,2021-01-01 00:00:00,Freelance,13239
3,2021-01-01 00:00:00,Food,6299
4,2021-01-01 00:00:00,Entertainment,2287


# Cleaning & Modelling the 'amount' column 

In [25]:
import re

def clean_amount_column(data):

    # Convert to string
    data["amount"] = data["amount"].astype(str)

    # Remove everything except numbers and decimal points
    data["amount"] = data["amount"].apply(lambda x: re.sub(r'[^\d.]', '', x))

    # Convert to numeric
    data["amount"] = pd.to_numeric(data["amount"], errors='coerce')

    # Replace NaN with 0 
    data["amount"] = data["amount"].fillna(0)

    return data


In [27]:
# Clean the 'amount' column
data = clean_amount_column(data)

print(data)

                      date       category    amount
0      2021-01-01 00:00:00      Education    3888.0
1      2021-01-01 00:00:00           Rent     649.0
2      2021-01-01 00:00:00      Freelance   13239.0
3      2021-01-01 00:00:00           Food    6299.0
4      2021-01-01 00:00:00  Entertainment    2287.0
...                    ...            ...       ...
15895                  NaN           Rent    7335.0
15896                  NaN      Education    4353.0
15897                  NaN           Food    1048.0
15898                  NaN         Others  621176.0
15899                  NaN      Utilities    5459.0

[15900 rows x 3 columns]


# Formatting the 'date' column

In [30]:
def format_and_sort_date(data):

    data["date"] = pd.to_datetime(data["date"], errors='coerce', dayfirst=False)
    
    data = data.dropna(subset=["date"])
    
    data = data.sort_values(by="date")
    
    data["date"] = data["date"].dt.strftime("%d/%m/%Y")
    
    data = data.reset_index(drop=True)
    
    return data


In [32]:
# Apply function
data = format_and_sort_date(data)

print(data)

             date       category      amount
0      01/01/2021      Education   3888.0000
1      01/01/2021      Utilities   5767.0000
2      01/01/2021      Utilities   4262.0000
3      01/01/2021         Others  59543.0000
4      01/01/2021      Utilities   5070.0000
...           ...            ...         ...
15409  31/12/2024           Rent   5681.0000
15410  31/12/2024  Entertainment   7085.0000
15411  31/12/2024           Rent      0.7882
15412  31/12/2024           Rent  13000.0000
15413  31/12/2024     Investment  55216.0000

[15414 rows x 3 columns]


# Cleaning the data to remove irrelevant entries

In [35]:
data.isna().sum()

date        0
category    0
amount      0
dtype: int64

In [37]:
data = data.dropna()

In [39]:
data.shape

(15414, 3)

# Final Overview and EDA

In [42]:
data.head(10)

,date,category,amount
0,01/01/2021,Education,3888.0
1,01/01/2021,Utilities,5767.0
2,01/01/2021,Utilities,4262.0
3,01/01/2021,Others,59543.0
4,01/01/2021,Utilities,5070.0
5,01/01/2021,Salary,62061.0
6,01/01/2021,Utilities,999999999.0
7,01/01/2021,Food,4168.0
8,01/01/2021,Entertainment,2287.0
9,01/01/2021,Food,6299.0


In [44]:
data.shape

(15414, 3)

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15414 entries, 0 to 15413
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      15414 non-null  object 
 1   category  15414 non-null  object 
 2   amount    15414 non-null  float64
dtypes: float64(1), object(2)
memory usage: 361.4+ KB


In [48]:
data.describe

<bound method NDFrame.describe of              date       category      amount
0      01/01/2021      Education   3888.0000
1      01/01/2021      Utilities   5767.0000
2      01/01/2021      Utilities   4262.0000
3      01/01/2021         Others  59543.0000
4      01/01/2021      Utilities   5070.0000
...           ...            ...         ...
15409  31/12/2024           Rent   5681.0000
15410  31/12/2024  Entertainment   7085.0000
15411  31/12/2024           Rent      0.7882
15412  31/12/2024           Rent  13000.0000
15413  31/12/2024     Investment  55216.0000

[15414 rows x 3 columns]>

# Saving in .csv

In [51]:
data.to_csv("cleaned_data.csv", index=False)